In [ ]:
import os
import jsonlines
from tqdm import tqdm

base_dir = '/srv/elkhyo/lexquo/generation/basement/echr_qa'

for root, dirs, files in os.walk(base_dir):
    if 'generations' in dirs:
        generations_dir = os.path.join(root, 'generations')
        for file_name in tqdm(os.listdir(generations_dir), desc="Processing files"):
            if file_name.endswith('.jsonl'):
                file_path = os.path.join(generations_dir, file_name)
                with jsonlines.open(file_path, mode='r') as reader:
                    records = list(reader)
                
                for record in records:
                    if 'meta' in record and 'docid' in record['meta']:
                        record['meta']['docid'] = str(record['meta']['docid'])
                        
                with jsonlines.open(file_path, mode='w') as writer:
                    writer.write_all(records)

In [ ]:
from datasets import load_dataset, Value


split = "test"
workshop_hf_name = "ylkhayat/{dataset_name}-generation-workshop"
dataset_repo_name_prefix = "ECHR_QA"
workshop_hf_name = workshop_hf_name.format(dataset_name=dataset_repo_name_prefix)
setups = [
    "data",
    "bm25_noisy_oracle_passages_oracle_documents",
    "bm25_oracle_passages_oracle_documents",
    "bm25_relevant_passages_oracle_documents"
    ]
for setup in setups:
    url=f"https://huggingface.co/datasets/{workshop_hf_name}/resolve/main/{setup}/"
    current_dataset = load_dataset("parquet", data_files={split: f"{url}{split}*.parquet"})[split]
    current_dataset = current_dataset.cast_column("docid", Value(dtype='string', id=None))
    current_dataset = current_dataset.push_to_hub(workshop_hf_name, data_dir=setup)
